<a href="https://colab.research.google.com/github/sfnesbit/CSC466-Team13/blob/main/CSC_466_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 466 Team 13 Food Image Classification
Ty Farris | Marine Cossoul | Sean Nesbit

[Multiclass image classification article](https://towardsdatascience.com/end-to-end-pipeline-for-setting-up-multiclass-image-classification-for-data-scientists-2e051081d41c)

[Also](https://medium.com/swlh/convolutional-neural-networks-for-multiclass-image-classification-a-beginners-guide-to-6dbc09fabbd)

##  Mount drive
*You may need to visit CSC466 folder, and 'Add Shortcut to Drive' or 'Move To > My Drive"*

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
datadir = "drive/MyDrive/CSC466/csc466_data/"
testdir = datadir+'images_test/'
traindir = datadir+'images_train/'

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/CSC466/csc466_data

food_c101_n1000_r384x384x3.h5	    food_test_c101_n1000_r64x64x1.h5
food_c101_n10099_r32x32x1.h5	    food_test_c101_n1000_r64x64x3.h5
food_c101_n10099_r32x32x3.h5	    images_npy
food_c101_n10099_r64x64x1.h5	    images_test
food_c101_n10099_r64x64x3.h5	    images_train
food_test_c101_n1000_r128x128x1.h5  meta
food_test_c101_n1000_r128x128x3.h5  X.txt
food_test_c101_n1000_r32x32x1.h5    y.txt
food_test_c101_n1000_r32x32x3.h5


In [ ]:
import shutil
import os

dirs = [dir for dir in os.listdir(traindir) if os.path.isdir(traindir+dir)]
found = 0
count = 1
for dir in dirs:
  #print(count)
  count += 1
  dircount = 0
  for filename in os.listdir(traindir + dir):
    dircount += 1
    found += 1
  print(dircount)
print(f'Found {found} files')

#  Create train and test datasets

### Create Training .npy files

In [ ]:
import os
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display
import cv2 as cv
from matplotlib import pyplot as plt

image_train_path = datadir+"images_train"
x_train = []
y_train = []

count = 0
filesfound = 0
directories=[d for d in os.listdir(image_train_path) if os.path.isdir(image_train_path+'/'+d)]

for dir in directories:
  count = count + 1
  dircount = 0
  print("directory: {} | {}".format(count, dir))
  f = IntProgress(min=0, max=200) # instantiate the bar
  display(f) # display the bar

  dir_path = image_train_path + "/" + dir
  for file in os.listdir(dir_path):
    
    if dircount >= 200:
      break
    f.value += 1 # signal to increment the progress bar
    dircount += 1
    #read in the image
    imagePath = image_train_path + "/" + dir + "/" + file
    image = cv.imread(imagePath)

    #resize the image to 128x128
    image = cv.resize(image, (128, 128)) 

    #add the image and corresponding label to x_train and y_train
    x_train.append(image)
    y_train.append(dir)
    filesfound += 1
  
  #save results to a file
  x_train = np.array(x_train, dtype="float") / 255.0
  np.save(datadir+'images_npy' + f"/x_{dir}.npy", x_train)
  y_train = np.array(y_train)
  np.save(datadir+'images_npy' + f"/y_{dir}.npy", y_train)
  print(f'Writing file: {dir} | Found {filesfound} images, {len(x_train)} elements')
  #reset arrays
  filesfound = 0
  x_train = []
  y_train = []



# Load Train/Test data from saved .npy files

In [ ]:
import numpy as np
import os
from ipywidgets import IntProgress
from IPython.display import display

x,y = None, None
count = 1
filecount = 0
temp = None
split_count = 70
f = IntProgress(min=0, max=202) # instantiate the bar
display(f) # display the bar

for file in os.listdir(datadir+'images_npy/'):
  f.value += 1
  if "x_" in file:
    temp = np.load(datadir+'images_npy/'+file)[:split_count]
    if x is None:
      x = temp
    else:
      x = np.append(x,temp,0)
    
  elif 'y_' in file:
    temp = np.load(datadir+'images_npy/'+file)[:split_count]
    if y is None:
      y = temp
    else:
      y = np.append(y,temp, 0)
print(len(x),len(y))

IntProgress(value=0, max=202)

7070 7070


## Shuffle and split into train/test

In [ ]:
from sklearn.utils import shuffle
x,y = shuffle(x,y)

x_train, y_train, x_test, y_test = [],[],[],[]
split = 0.8
val_split = 0.9
train_test_split_index = int(np.ceil(split*len(x)))
test_val_split_index = int(np.ceil(val_split*len(x)))


x_train = x[:train_test_split_index]
y_train = y[:train_test_split_index]
x_test  = x[train_test_split_index:test_val_split_index]
y_test  = y[train_test_split_index:test_val_split_index]
x_val   = x[test_val_split_index:]
y_val   = y[test_val_split_index:]

print(f'Train: ({len(x_train)}, {len(y_train)}) | Test: ({len(x_test)}, {len(y_test)}) | Val: ({len(x_val)}),({len(y_val)})')


Train: (5656, 5656) | Test: (707, 707) | Val: (707),(707)


In [ ]:
ldict = {key: 0 for key in np.unique(y_train)}
for item in y_train:
  ldict[item] += 1

#creates a label mapping
labels = list(set(np.unique(y_train)).union(np.unique(y_test)))
labels = dict([(image_label.lower().replace(" ", "_"), index) for index, image_label in enumerate(labels)])
print(labels)

{'omelette': 0, 'lobster_roll_sandwich': 1, 'seaweed_salad': 2, 'dumplings': 3, 'pulled_pork_sandwich': 4, 'sashimi': 5, 'pizza': 6, 'huevos_rancheros': 7, 'breakfast_burrito': 8, 'french_toast': 9, 'pancakes': 10, 'prime_rib': 11, 'onion_rings': 12, 'foie_gras': 13, 'chicken_quesadilla': 14, 'tiramisu': 15, 'tuna_tartare': 16, 'spaghetti_carbonara': 17, 'garlic_bread': 18, 'strawberry_shortcake': 19, 'grilled_cheese_sandwich': 20, 'beef_tartare': 21, 'fried_rice': 22, 'baklava': 23, 'bibimbap': 24, 'tacos': 25, 'chicken_curry': 26, 'risotto': 27, 'clam_chowder': 28, 'mussels': 29, 'guacamole': 30, 'fish_and_chips': 31, 'scallops': 32, 'beet_salad': 33, 'falafel': 34, 'frozen_yogurt': 35, 'ravioli': 36, 'gyoza': 37, 'carrot_cake': 38, 'sushi': 39, 'baby_back_ribs': 40, 'eggs_benedict': 41, 'cup_cakes': 42, 'deviled_eggs': 43, 'shrimp_and_grits': 44, 'miso_soup': 45, 'hot_dog': 46, 'hot_and_sour_soup': 47, 'ceviche': 48, 'edamame': 49, 'greek_salad': 50, 'hummus': 51, 'beignets': 52, 'c

In [ ]:
from keras.utils.np_utils import to_categorical   

#one hot encoded label values
categorical_labels = to_categorical(np.arange(101), num_classes=101)
categorical_labels

y_train2 = []
y_test2 = []
y_val2 = []
for i in range(len(y_train)):
  label_index = labels[y_train[i]]
  y_train2.append(categorical_labels[label_index])
  
for i in range(len(y_test)):
  label_index = labels[y_test[i]]
  y_test2.append(categorical_labels[label_index])
  
for i in range(len(y_val)):
  label_index = labels[y_val[i]]
  y_val2.append(categorical_labels[label_index])

# Build model 

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(101, activation='softmax'))


In [ ]:
# finally compile and train the cnn
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
cnn.fit(x=x_train, y=np.array(y_train2), validation_data=(x_val, np.array(y_val2)), batch_size=32, epochs=20, verbose=True)

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

history = cnn.fit(x_train, np.array(y_train2), batch_size=32, epochs=20, validation_data=(x_val,np.array(y_val2)), verbose=True)

Epoch 1/20
177/177 [==============================] - 4s 19ms/step - loss: 4.6170 - accuracy: 0.0104 - val_loss: 4.6164 - val_accuracy: 0.0071
Epoch 2/20
177/177 [==============================] - 3s 17ms/step - loss: 4.6156 - accuracy: 0.0115 - val_loss: 4.6177 - val_accuracy: 0.0071
Epoch 3/20
177/177 [==============================] - 3s 17ms/step - loss: 4.6154 - accuracy: 0.0095 - val_loss: 4.6186 - val_accuracy: 0.0071
Epoch 4/20
177/177 [==============================] - 3s 17ms/step - loss: 4.6156 - accuracy: 0.0092 - val_loss: 4.6206 - val_accuracy: 0.0071
Epoch 5/20
177/177 [==============================] - 3s 17ms/step - loss: 4.6155 - accuracy: 0.0111 - val_loss: 4.6210 - val_accuracy: 0.0071
Epoch 6/20
177/177 [==============================] - 3s 17ms/step - loss: 4.6152 - accuracy: 0.0094 - val_loss: 4.6214 - val_accuracy: 0.0071
Epoch 7/20
177/177 [==============================] - 3s 16ms/step - loss: 4.6149 - accuracy: 0.0099 - val_loss: 4.6219 - val_accuracy: 0.0071

KeyboardInterrupt: ignored

# Evaluate model

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('accuracy over time')
plt.legend(['train','val'])
plt.show()

In [ ]:
print(y_train[55],np.argmax(y_train2[55]))
#np.shape(x_train[0])
img = x_train[55].reshape(1,128,128,3)
np.argmax(cnn.predict(img))

deviled_eggs 35


35

In [ ]:

from ipywidgets import IntProgress
from IPython.display import display

f = IntProgress(min=0, max=707) # instantiate the bar
display(f) # display the bar
preds = {}
for i in range(len(x_val)):
  f.value += 1
  img = x_val[i].reshape(1,128,128,3)
  pred = np.argmax(cnn.predict(img))
  if pred in preds:
    preds[pred] += 1
  else:
    preds[pred] = 1
print(sorted(preds.items()))